# Import Libraries and Clone Github

In [ ]:
# Import Libraries
import os
from google.colab import userdata
import google.generativeai as genai

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

# Import Data

### NOTE: Updated text processing file to add unique identifier, and also concatenate the source to the identifier.  Make sure to separate the earnings presentations from the earnings calls also

In [ ]:
# Import all cleaned data files and merge into a df
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
df['Article Index'] = df.index
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date']= pd.to_datetime(df['Date']).dt.date
df

# Test Gemini Sentiment Analysis

In [ ]:
# Set up prompt

In [ ]:
# Set up function to query gemini

In [ ]:
# Find a random article from each category.  Maybe 3.  Print them out. Depending on size, they might have to be different cells

In [ ]:
# Query the articles and print the results for QC